In [72]:
import pandas as pd
from scipy.stats import mode

In [73]:
df = pd.io.pickle.read_pickle("labbDF.pkl")
df['name'] = df.apply(lambda row: row['name'].decode('latin-1'), axis=1)

In [87]:
""" Generate fallback map from municipalitys """
hierarchy = pd.io.excel.read_excel("hierarchy.xlsx")

word = "tipsrunda"

def getMuni(df, level, key):
    return df.groupby(level).get_group(key)['Kommun'].unique()

def getParent(df, municipality, level):
    try:
        parent = hierarchy.loc[hierarchy[u'Kommun'] == municipality][level].values[0]
        if not parent == "-":
            munis = getMuni(hierarchy, level, parent)
            return parent, mode(df.loc[df['name'].isin(munis)]['bins_'+word])[0][0]
        else:
            return None, None
    except IndexError:
        return None, None

def updateDF(df, parentLevel):
    """ Find municipalitys with no hits and update according to rule """
    for muni in df[df['bins_'+word] == -1]['name'].unique():
        parent, mean = getParent(df, muni, parentLevel)

        # Update municipality with fallback according to rule
        if mean and mean != -1:
            print df.loc[df['name'] == muni]
            df.loc[df['name'] == muni, 'bins_'+word] = mean # Empty dataframe! Unicode problem?
            print df.loc[df['name'] == muni]
            print muni, "->", muni, mean
            #print df.loc[df['name'] == key]
    return df 

In [88]:
print df.head()

     area            name                                               poly  \
0    75.4  Upplands Väsby  POLYGON ((1103922.593170376 1022495.837027514,...   
1   359.7      Vallentuna  POLYGON ((1137678.584388074 1024404.619503576,...   
2   310.0       Österåker  POLYGON ((1176139.12166001 1033496.310143271, ...   
4   438.5          Värmdö  POLYGON ((1164922.989820984 976939.4120838717,...   
12   50.7        Järfälla  POLYGON ((1090092.252866263 1014064.051743116,...   

    tipsrunda  bins_tipsrunda  
0    1.069948               2  
1    0.000000              -1  
2    0.464148               0  
4    0.000000              -1  
12   0.326204               0  


In [89]:
len(df[df['bins_tipsrunda'] == -1]['bins_tipsrunda'].values)

216

In [90]:
df[df['bins_tipsrunda'] == -1].head()

,area,name,poly,tipsrunda,bins_tipsrunda
1,359.7,Vallentuna,"POLYGON ((1137678.584388074 1024404.619503576,...",0,-1
4,438.5,Värmdö,"POLYGON ((1164922.989820984 976939.4120838717,...",0,-1
13,216.0,Ekerö,"POLYGON ((1088225.889478081 984642.190218294, ...",0,-1
14,216.0,Ekerö,"POLYGON ((1068917.783098555 992787.9296804154,...",0,-1
15,216.0,Ekerö,"POLYGON ((1054999.417345152 995986.786836545, ...",0,-1


In [91]:
hierarchy = pd.io.excel.read_excel("hierarchy.xlsx")

In [92]:
df = updateDF(df, u"Stadsomland")

     area     name                                               poly  \
46  311.5  Knivsta  POLYGON ((1068131.766109952 1074061.936456923,...   

    tipsrunda  bins_tipsrunda  
46          0              -1  
     area     name                                               poly  \
46  311.5  Knivsta  POLYGON ((1068131.766109952 1074061.936456923,...   

    tipsrunda  bins_tipsrunda  
46          0               2  
Knivsta -> Knivsta 2
      area          name                                               poly  \
113  107.7  Staffanstorp  POLYGON ((586633.3868181518 230874.8715049047,...   

     tipsrunda  bins_tipsrunda  
113          0              -1  
      area          name                                               poly  \
113  107.7  Staffanstorp  POLYGON ((586633.3868181518 230874.8715049047,...   

     tipsrunda  bins_tipsrunda  
113          0               2  
Staffanstorp -> Staffanstorp 2
     area   name                                               poly  \
120  

In [93]:
len(df[df['bins_tipsrunda'] == -1]['bins_tipsrunda'].values)

208

In [94]:
df.loc[df['name'] == "Lomma", 'bins_tipsrunda']

120    2
Name: bins_tipsrunda, dtype: int64